In [1]:
import nltk
#from nltk.tokenize import wordPunctTokenizer
import difflib
import pyodbc
import pandas as pd
import re


In [2]:
data1=[['flat no. 36,    vishal appartments, kishanghar, vasant kunj','WWW 111'],['house no. 33, haiderpur post sadar jaunpur','BBBB 666'],['fiserv sector 49 ','RG9 3DT']]
data2=[['flat no. 36,    vishal appartments, kishanghar, vasant kunj, new delhi &^%$','WWW 111'],['fiserv sector 49 gurugram','RRR 333'],['house no. 33, haiderpur post sadar jaunpur','BBBB 666']]

In [3]:
df1=pd.DataFrame(data1,columns=['Address','PostalCode'])
df2=pd.DataFrame(data2,columns=['Address','PostalCode'])

In [4]:
df1

,Address,PostalCode
0,"flat no. 36, vishal appartments, kishanghar...",WWW 111
1,"house no. 33, haiderpur post sadar jaunpur",BBBB 666
2,fiserv sector 49,RG9 3DT


In [5]:
#pip install textdistance
import textdistance

In [6]:
#Jaccard index
textdistance.ratcliff_obershelp(df1['Address'][0], df2['Address'][0])

0.8805970149253731

In [7]:
from nltk.corpus import stopwords
import string
import nltk

In [8]:
#nltk.download_shell()

In [9]:
def text_process(mess):

    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    abc=[word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    
    abc = ' '.join(abc)
    # Now just remove any stopwords
    return abc

In [10]:
df1['Address']=df1['Address'].apply(text_process)
df1

,Address,PostalCode
0,flat 36 vishal appartments kishanghar vasant kunj,WWW 111
1,house 33 haiderpur post sadar jaunpur,BBBB 666
2,fiserv sector 49,RG9 3DT


In [11]:
df2['Address']=df2['Address'].apply(text_process)
df2

,Address,PostalCode
0,flat 36 vishal appartments kishanghar vasant k...,WWW 111
1,fiserv sector 49 gurugram,RRR 333
2,house 33 haiderpur post sadar jaunpur,BBBB 666


In [12]:
#pip install textdistance
import textdistance

In [13]:
#Jaccard index
textdistance.ratcliff_obershelp(df1['Address'][0], df2['Address'][0])

0.9074074074074074

In [14]:
df1['Address'][0]

'flat 36 vishal appartments kishanghar vasant kunj'

In [41]:
x1=[]
Base_Name=[]
CompleteMatchTargetName=[]
PartialMatch=[]
df=pd.DataFrame()
for i in range(len(df1['Address'])):
    for j in range(len(df2['Address'])):
        x=textdistance.ratcliff_obershelp(df1['Address'][i], df2['Address'][j])
        if x==1:
            Base_Name.append(df1['Address'][i])
            CompleteMatchTargetName.append(df2['Address'][j])
            PartialMatch.append('')
            x1.append(x)
        elif x > .6:
            Base_Name.append(df1['Address'][i])
            PartialMatch.append(df2['Address'][j])
            CompleteMatchTargetName.append('')
            x1.append(x)
df['Base_Name']=Base_Name
df['matchpct']=x1
df['CompleteMatchTargetName']=CompleteMatchTargetName
df['PartialMatch']=PartialMatch

In [42]:
df

,Base_Name,matchpct,CompleteMatchTargetName,PartialMatch
0,flat 36 vishal appartments kishanghar vasant kunj,0.907407,,flat 36 vishal appartments kishanghar vasant k...
1,house 33 haiderpur post sadar jaunpur,1.000000,house 33 haiderpur post sadar jaunpur,
2,fiserv sector 49,0.780488,,fiserv sector 49 gurugram


In [44]:
    writer =pd.ExcelWriter('outputGeneric.xlsx', engine ='xlsxwriter')
    df.to_excel(writer, sheet_name='Sheet1')
    writer.save()